# Alive & Thrive
## Viet Nam Data: Child Files - Preprocessing
## Prepared by Aaron Wise; aaron@a3di.dev
### Version: 28 May 2022

In [17]:
# %load std_imports.py
from pathlib import Path

import numpy as np
import pandas as pd

import json
import pyreadstat

from std_utils import (
    read_spss_file,
    run_quality_assurance,
    generate_HHID,
    generate_MEMID,
    subset_edu_save,
    add_total_year,
    merge_hh_hl_data,
    subset_hl_df,
    standardize_col_names,
    standardize_col_values,
    create_elderly_hoh,
    save_merge,
    export_analyzed_data
)

pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", None)


In [18]:
# %load children_imports.py
from children_analysis import (
    generate_MOMID,
    merge_mother_edu,
    create_sex_ch,
    create_ch_age_cat,
    create_excl_bf,
    create_cont_1223_bf,
    create_mdd_ch,
    create_mmf_ch,
    create_mad_ch
)

from aw_analytics import mean_wt, output_mean_table

### --- 2021 ---

In [19]:
# Set parameters
country = 'VNM'
recode = 'children'

# -------------------------------------------------------------------
year = '2021'
# -------------------------------------------------------------------

In [20]:
# Read file
df = read_spss_file(country, year, recode)

# Create HHID to facilitate merge of HH and HL data
generate_HHID(df, country, year, recode)

# Create WMID to facilitate merge into U5 file
generate_MOMID(df, country, year)

# Add Total, Year
add_total_year(df, year)

# Run quality assurance
run_quality_assurance(df)

# Merge in HH and HL data
df = merge_hh_hl_data(df, country, year)

# Merge in Mother Education data
df = merge_mother_edu(df, country, year)

The file -- ch_2021.sav -- has the following shape: Rows: 4404; Columns: 537
HHID is NOT unique
MOMID is NOT unique
Drop columns if all values are NaN...
Updated -- Rows: 4404; Columns: 537
Checking if any rows are duplicates...
The are no duplicate rows


In [21]:
df.head(5)

,HH1,HH2,LN,UF1,UF2,UF3,UFINT,UF4,UF5,UF6,UF7D,UF7M,UF7Y,UF9,UF10,UF17,UF8H,UF8M,UF11H,UF11M,UF12,UF13,UF14,UF15,UF21,UF22,UF23,UFHINT,CHFIN,UB1D,UB1M,UB1Y,UB2,UB6,UB7,UB8,UB9,UB10B,UB10C,UB10D,UB10X,UB10NR,BR1,BR2,BR3,EC1,EC2A,EC2B,EC2C,EC3A,EC3B,EC5AA,EC5AB,EC5AX,EC5AY,EC5ANR,EC5BA,EC5BB,EC5BX,EC5BY,EC5BNR,EC5CA,EC5CB,EC5CX,EC5CY,EC5CNR,EC5DA,EC5DB,EC5DX,EC5DY,EC5DNR,EC5EA,EC5EB,EC5EX,EC5EY,EC5ENR,EC5FA,EC5FB,EC5FX,EC5FY,EC5FNR,EC21,EC22,EC23,EC24,EC25,EC26,EC27,EC28,EC29,EC30,EC31,EC32,EC33,EC34,EC35,EC36,EC37,EC38,EC39,EC40,UCD2A,UCD2B,UCD2C,UCD2D,UCD2E,UCD2F,UCD2G,UCD2H,UCD2I,UCD2J,UCD2K,UCD3,UCD4,UCD5,UCF2,UCF3,UCF4,UCF7,UCF9,UCF11,UCF12,UCF13,UCF14,UCF15,UCF16,UCF17,UCF18,BD2,BD3,BD4,BD5,BD6,BD7A,BD7B1,BD7B2,BD7C,BD7D,BD7D1,BD7E,BD7E1,BD7X,BD8A,BD8A1,BD8B,BD8C,BD8D,BD8E,BD8F,BD8G,BD8H,BD8I,BD8J,BD8K,BD8L,BD8M,BD8N,BD8X,BD9,BD8DUMMY,IM2,IM3,IM5,IM6BD,IM6BM,IM6BY,IM6P1D,IM6P1M,IM6P1Y,IM6P2D,IM6P2M,IM6P2Y,IM6P3D,IM6P3M,IM6P3Y,IM6I1D,IM6I1M,IM6I1Y,IM6I2D,IM6I2M,IM6I2Y,IM6I3D,IM6I3M,IM6I3Y,IM6ID,IM6IM,IM6IY,IM6DPT1D,IM6DPT1M,IM6DPT1Y,IM6DPT2D,IM6DPT2M,IM6DPT2Y,IM6DPT3D,IM6DPT3M,IM6DPT3Y,IM6DPT4D,IM6DPT4M,IM6DPT4Y,IM6HEPB0D,IM6HEPB0M,IM6HEPB0Y,IM6HEPB1D,IM6HEPB1M,IM6HEPB1Y,IM6HEPB2D,IM6HEPB2M,IM6HEPB2Y,IM6HEPB3D,IM6HEPB3M,IM6HEPB3Y,IM6HIB1D,IM6HIB1M,IM6HIB1Y,IM6HIB2D,IM6HIB2M,IM6HIB2Y,IM6HIB3D,IM6HIB3M,IM6HIB3Y,IM6PCV1D,IM6PCV1M,IM6PCV1Y,IM6PCV2D,IM6PCV2M,IM6PCV2Y,IM6PCV3D,IM6PCV3M,IM6PCV3Y,IM6M1D,IM6M1M,IM6M1Y,IM6M2D,IM6M2M,IM6M2Y,IM9,IM11,IM12,IM14,IM15,IM16,IM18,IM19,IM20,IM21,IM21B,IM21C,IM21D,IM21E,IM21G,IM21H,IM22A,IM22B,IM26,IM26A,IM26B,IM26C,CA1,CA3,CA4,CA5,CA6A,CA6B,CA6C,CA6D,CA6E,CA6F,CA6H,CA6I,CA6J,CA6K,CA6O,CA6W,CA6P,CA6Q,CA6R,CA6X,CA6Z,CA6NR,CA7A,CA7B,CA7C,CA7D,CA9A,CA9B,CA9C,CA9D,CA9E,CA9F,CA9H,CA9I,CA9J,CA9K,CA9O,CA9W,CA9P,CA9Q,CA9R,CA9X,CA9Z,CA9NR,CA11A,CA11B,CA11C,CA11D,CA11E,CA11F,CA11H,CA11I,CA11J,CA11K,CA11O,CA11W,CA11P,CA11Q,CA11R,CA11X,CA11Z,CA11NR,CA12,CA13A,CA13B,CA13G,CA13H,CA13L,CA13M,CA13N,CA13O,CA13Q,CA13X,CA13NR,CA14,CA16,CA17,CA18,CA20,CA21A,CA21B,CA21C,CA21D,CA21E,CA21F,CA21H,CA21I,CA21J,CA21K,CA21O,CA21W,CA21P,CA21Q,CA21R,CA21X,CA21Z,CA21NR,CA22,CA23L,CA23M,CA23N,CA23O,CA23R,CA23S,CA23T,CA23W,CA23X,CA23Z,CA23NR,CA25A,CA25B,CA25C,CA25D,CA25E,CA25F,CA25H,CA25I,CA25J,CA25K,CA25O,CA25W,CA25P,CA25Q,CA25R,CA25X,CA25Z,CA25NR,CA31,UF24$1,UF24$2,UF24$3,UF25A$1,UF25A$2,UF25A$3,UF25B$1,UF25B$2,UF25B$3,UF26A$1,UF26A$2,UF26A$3,UF26B$1,UF26B$2,UF26B$3,UF26C$1,UF26C$2,UF26C$3,UF26D$1,UF26D$2,UF26D$3,UF26E$1,UF26E$2,UF26E$3,UF26X$1,UF26X$2,UF26X$3,UF27$1,UF27$2,UF27$3,HF1,HF2,HF3,HF4,HF5,HF6,HF7D,HF7M,HF7Y,HF8H,HF8M,HF14H,HF14M,HF9D,HF9M,HF9Y,HF15,HF11D,HF11M,HF11Y,HF12BD,HF12BM,HF12BY,HF12P1D,HF12P1M,HF12P1Y,HF12P2D,HF12P2M,HF12P2Y,HF12P3D,HF12P3M,HF12P3Y,HF12I1D,HF12I1M,HF12I1Y,HF12I2D,HF12I2M,HF12I2Y,HF12I3D,HF12I3M,HF12I3Y,HF12ID,HF12IM,HF12IY,HF12DPT1D,HF12DPT1M,HF12DPT1Y,HF12DPT2D,HF12DPT2M,HF12DPT2Y,HF12DPT3D,HF12DPT3M,HF12DPT3Y,HF12DPT4D,HF12DPT4M,HF12DPT4Y,HF12HEPB0D,HF12HEPB0M,HF12HEPB0Y,HF12HEPB1D,HF12HEPB1M,HF12HEPB1Y,HF12HEPB2D,HF12HEPB2M,HF12HEPB2Y,HF12HEPB3D,HF12HEPB3M,HF12HEPB3Y,HF12HIB1D,HF12HIB1M,HF12HIB1Y,HF12HIB2D,HF12HIB2M,HF12HIB2Y,HF12HIB3D,HF12HIB3M,HF12HIB3Y,HF12PCV1D,HF12PCV1M,HF12PCV1Y,HF12PCV2D,HF12PCV2M,HF12PCV2Y,HF12PCV3D,HF12PCV3M,HF12PCV3Y,HF12M1D,HF12M1M,HF12M1Y,HF12M2D,HF12M2M,HF12M2Y,HH4,HH6,HH7,HH7A,HL4,ED5A_x,ED5B,CDOI,CDOB,CAGE,CAGE_6,CAGE_11,CAGED,melevel,cinsurance,cdisability,ethnicity,ethnicity2,chweight,wscore,windex5,windex10,wscoreu,windex5u,windex10u,wscorer,windex5r,windex10r,PSU,stratum,HHID,MOMID,Total,Year,residence,region,wealth_q,eth_hoh,elderly_hoh,sex_hoh,MEMID,ED5A_y
0,1.0,4.0,4.0,1.0,4.0,4.0,92.0,002,92.0,91.0,18.0,11.0,2020.0,"YES, INTERVIEWED ALREADY",YES,COMPLETED,12.0,8.0,12.0,18.0,VIETNAMESE,VIETNAMESE,VIETNAMESE,"NO, NOT USED",NaN,NaN,NaN,94.0,SAVE QUESTIONNAIRE AND FINISH,13.0,JANUARY,2016.0,4.0,NO,NaN,NaN,YES,,COVERED BY GOVERNMENT,,,,"YES, SEEN",NaN,NaN,TEN OR MORE BOOKS,YES,YES,YES,NONE,NONE,MOTHER,,OTHER,,,MOTHER,,OTHER,,,MOTHER,,OTHER,,,MOTHER

#### Create Indicators

In [22]:
# --- Child Sex [sex_ch] --- #
df = create_sex_ch(df, country, year)

In [23]:
# --- Age categories [ch_age_cat_X] --- #
df = create_ch_age_cat(df, country, year)

In [24]:
# --- Exclusive BF [excl_bf] --- #
df = create_excl_bf(df, country, year)

# --- Continued Breastfeeding 12-23 mos [cont_1223_bf] --- #
df = create_cont_1223_bf(df, country, year)

# --- Minimum Dietary Diversity [mdd_ch] --- #
df = create_mdd_ch(df, country, year)

# --- Minimum Meal Frequency [mmf_ch] --- #
df = create_mmf_ch(df, country, year)

# --- Minimum Acceptable Diet [mad_ch] --- #
df = create_mad_ch(df)

In [ ]:
df.mother_edu.value_counts(dropna=False)

In [25]:
var = 'mad_ch'
ind_var = ['melevel']
weight = ['chweight']

output_mean_table(df, var, ind_var, weight)

,mad_ch,Weighted_Count
melevel,,
Lower Secondary,35.3,1235.2
Pre-primary or none,18.0,168.4
Primary,30.6,348.2
University/College/Higher,54.5,1205.3
Upper Secondary,51.6,1078.2
Vocational High school,48.8,293.7


#### Create Equity Stratifiers

In [ ]:
# --- Child Sex --- #
df['female'] = np.where(df['HL4'] == 'FEMALE', 'Female', 'Male')


#### Subset and export working dataset